<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FINAL_ACP_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/agentic-commerce-protocol/agentic-commerce-protocol.git

In [1]:
#!git clone https://github.com/agentic-commerce-protocol/agentic-commerce-protocol.git

# NEW: Import OpenAI for LLM integration
from openai import OpenAI

import requests
import json
import uuid
from datetime import datetime, timezone
import copy
import sys

# ==============================================================================
# 0. CONFIGURATION AND EXTERNAL DATA LOADING
# ==============================================================================

# --- ACTUAL GITHUB PATHS (Used for simulated fetching) ---
ACP_GITHUB_BASE = "https://raw.githubusercontent.com/agentic-commerce-protocol/agentic-commerce-protocol/main/examples"
AGENTIC_CHECKOUT_URL = f"{ACP_GITHUB_BASE}/examples.agentic_checkout.json"
DELEGATE_PAYMENT_URL = f"{ACP_GITHUB_BASE}/examples/examples.delegate_payment.json"

# API Endpoints & Auth
BASE_URL = "https://merchant.example.com"
PAYMENT_PROVIDER_URL = "https://payment.provider.com/tokens"
CHECKOUT_SESSION_ID = "csn_01HV3P3XYZ9ABC"
API_BEARER_TOKEN = "api_key_123"

# --- MOCK DATA TO SIMULATE FETCHING THE EXTERNAL JSON FILES ---
# This dictionary holds the content that a live `requests.get(URL).json()` would return.
EXTERNAL_JSON_CONTENT = {
    "create_checkout_session_request": {"items": [{"id": "item_123", "quantity": 1}], "fulfillment_address": {"name": "John Doe", "line_one": "1234 Chat Road,", "line_two": "", "city": "San Francisco", "state": "CA", "country": "US", "postal_code": "94131"}},
    "create_checkout_session_response": {"id": CHECKOUT_SESSION_ID, "payment_provider": {"provider": "stripe", "supported_payment_methods": ["card"]}, "status": "ready_for_payment", "currency": "usd", "line_items": [{"id": "line_item_123", "item": {"id": "item_123", "quantity": 1}, "base_amount": 300, "discount": 0, "subtotal": 300, "tax": 30, "total": 330}], "fulfillment_address": {"name": "John Doe", "line_one": "1234 Chat Road,", "line_two": "", "city": "San Francisco", "state": "CA", "country": "US", "postal_code": "94131"}, "fulfillment_option_id": "fulfillment_option_123", "totals": [{"type": "total", "display_text": "Total", "amount": 430}, {"type": "fulfillment", "display_text": "Fulfillment", "amount": 100}], "fulfillment_options": [{"type": "shipping", "id": "fulfillment_option_123", "title": "Standard", "subtotal": 100, "total": 100}, {"type": "shipping", "id": "fulfillment_option_456", "title": "Express", "subtotal": 500, "total": 500}], "messages": [], "links": []},
    "update_checkout_session_request": {"fulfillment_option_id": "fulfillment_option_456"},
    "update_checkout_session_response": {"id": CHECKOUT_SESSION_ID, "payment_provider": {"provider": "stripe", "supported_payment_methods": ["card"]}, "status": "ready_for_payment", "currency": "usd", "line_items": [{"id": "line_item_123", "item": {"id": "item_123", "quantity": 1}, "base_amount": 300, "discount": 0, "subtotal": 300, "tax": 30, "total": 330}], "fulfillment_address": {"name": "John Doe", "line_one": "1234 Chat Road,", "line_two": "", "city": "San Francisco", "state": "CA", "country": "US", "postal_code": "94131"}, "fulfillment_option_id": "fulfillment_option_456", "totals": [{"type": "total", "display_text": "Total", "amount": 830}, {"type": "fulfillment", "display_text": "Fulfillment", "amount": 500}], "fulfillment_options": [{"type": "shipping", "id": "fulfillment_option_123", "title": "Standard", "subtotal": 100, "total": 100}, {"type": "shipping", "id": "fulfillment_option_456", "title": "Express", "subtotal": 500, "total": 500}], "messages": [], "links": []},
    "delegate_payment_request": {"payment_method": {"type": "card", "number": "4242..."}, "allowance": {"checkout_session_id": CHECKOUT_SESSION_ID, "merchant_id": "acme_store"}, "billing_address": {"name": "Ada Lovelace", "line_one": "1234 Chat Road"}},
    "delegate_payment_success_response": {"id": "spt_SECURETOKEN", "created": "2025-09-29T11:00:00Z", "metadata": {"source": "agent_checkout", "merchant_id": "acme_store"}},
    "complete_checkout_session_request_template": {"buyer": {"first_name": "Jane"}, "payment_data": {"token": "", "provider": "stripe"}},
    "complete_checkout_session_response": {"id": CHECKOUT_SESSION_ID, "status": "completed", "totals": [{"type": "total", "display_text": "Total", "amount": 830}], "order": {"id": "order_123", "checkout_session_id": CHECKOUT_SESSION_ID, "permalink_url": "https://www.testshop.com/orders/order_123"}}
}

# This dictionary will hold the final merged, usable data
ACP_DATA = {}

def simulate_data_fetch(url):
    """
    Simulates a live HTTP GET request to the specified GitHub URL.
    In a live environment, this would be: requests.get(url).json()
    """
    # Use a key to fetch data content from the simplified mock source
    if "agentic_checkout.json" in url:
        # Simulate merging all checkout-related examples
        return {k: v for k, v in EXTERNAL_JSON_CONTENT.items() if k.startswith(('create', 'update', 'complete'))}
    elif "delegate_payment.json" in url:
        # Simulate merging all delegate-related examples
        return {k: v for k, v in EXTERNAL_JSON_CONTENT.items() if k.startswith('delegate')}
    else:
        raise FileNotFoundError(f"Simulated file not found for URL: {url}")

# --- Load the data structure using the specified paths ---
try:
    print("--- Initializing Data Structure ---")
    checkout_data = simulate_data_fetch(AGENTIC_CHECKOUT_URL)
    delegate_data = simulate_data_fetch(DELEGATE_PAYMENT_URL)

    ACP_DATA.update(checkout_data)
    ACP_DATA.update(delegate_data)

    print(f"[SUCCESS] Loaded data from {AGENTIC_CHECKOUT_URL} and {DELEGATE_PAYMENT_URL}")
    print(f"[SUCCESS] Data Keys available: {list(ACP_DATA.keys())}")
    print("---------------------------------")
except Exception as e:
    print(f"FATAL ERROR: Could not load data from simulated GitHub paths: {e}")
    sys.exit(1)


# --- MOCKING & UTILITY FUNCTIONS ---

def get_headers(step_id="general"):
    """Generates the required dynamic and static headers for the ACP calls."""
    key_prefix = f"acp_demo_{step_id}_{str(uuid.uuid4())}"
    return {"Authorization": f"Bearer {API_BEARER_TOKEN}", "Content-Type": "application/json", "API-Version": "2025-09-29", "Idempotency-Key": f"{key_prefix}_idem"}

class MockResponse:
    # Simulates the HTTP response object
    def __init__(self, json_data, status_code, url, method):
        self.status_code = status_code
        self.reason = "Created" if status_code == 201 else ("OK" if status_code < 400 else "ERROR")
        self.request = type('MockRequest', (object,), {'url': url, 'method': method})()
        self._json = json_data
    def json(self): return self._json
    def raise_for_status(self):
        if self.status_code >= 400: raise requests.exceptions.HTTPError(f"{self.status_code} {self.reason}")

def mock_api_call(method, url, payload, step_name):
    # Simulates the API execution and returns the expected mock response
    if step_name == "CREATE": return MockResponse(ACP_DATA["create_checkout_session_response"], 201, url, method)
    elif step_name == "UPDATE": return MockResponse(ACP_DATA["update_checkout_session_response"], 200, url, method)
    elif step_name == "DELEGATE":
        resp = copy.deepcopy(ACP_DATA["delegate_payment_success_response"])
        resp['id'] = f"spt_{uuid.uuid4().hex[:10].upper()}" # Dynamic SPT generation
        return MockResponse(resp, 200, url, method)
    elif step_name == "COMPLETE": return MockResponse(ACP_DATA["complete_checkout_session_response"], 200, url, method)
    else: return MockResponse({"error": "Unknown step"}, 500, url, method)

def print_call_info(step_title, response, payload):
    # Prints the structured information for clarity
    print(f"\n====================== {step_title} ======================")
    print(f"Endpoint: {response.request.method} {response.request.url}")
    print(f"Status: {response.status_code} {response.reason}")
    print("--- REQUEST BODY ---")
    print(json.dumps(payload, indent=2))
    print("--- RESPONSE BODY (Full Authoritative State) ---")
    print(json.dumps(response.json(), indent=2))
    print("====================================================")
    return response.json()


from google.colab import userdata
from openai import OpenAI

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)


# NEW: Function to query GPT-3.5-turbo for decision-making
def query_llm(prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=100):

    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    #client = OpenAI()
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an intelligent agent helping with e-commerce decisions. Respond concisely with only the requested output."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"LLM Error: {e}")
        return None  # Fallback to default if API fails

# ==============================================================================
# MAIN STRUCTURED ACP TRANSACTION FLOW (WITH LLM LOGIC)
# ==============================================================================

print("\n--- Agentic Commerce Protocol (ACP) Structured Transaction Flow ---")

SESSION_ID = CHECKOUT_SESSION_ID
SPT_TOKEN = None

# 1. CREATE CHECKOUT SESSION (POST /checkout_sessions)
step_name = "CREATE"
url = f"{BASE_URL}/checkout_sessions"
payload = ACP_DATA["create_checkout_session_request"]
response = mock_api_call("POST", url, payload, step_name)
session_data = print_call_info("1. CREATE SESSION (Initialize Cart)", response, payload)

# NEW: Use LLM to decide on shipping option (simulating user preference)
# Example user preference: "I want fast delivery." (You can make this dynamic/input-based)
user_preference = "I want fast delivery."  # Change this to test different preferences
fulfillment_options = session_data["fulfillment_options"]
options_str = ", ".join([f"{opt['id']} ({opt['title']}, ${opt['total']/100})" for opt in fulfillment_options])
prompt = f"Based on the user's preference: '{user_preference}'. Choose the best shipping option ID from: {options_str}. Output only the ID (e.g., fulfillment_option_456)."
llm_choice = query_llm(prompt)
print(f"\n[LLM DECISION] Selected fulfillment option: {llm_choice} (based on preference: '{user_preference}')")

# ------------------------------------------------------------------------------
# 2. UPDATE CHECKOUT SESSION (Select Shipping) (POST /checkout_sessions/{id})
# ------------------------------------------------------------------------------
step_name = "UPDATE"
url = f"{BASE_URL}/checkout_sessions/{SESSION_ID}"
payload = {"fulfillment_option_id": llm_choice if llm_choice else "fulfillment_option_456"}  # Fallback to Express if LLM fails
response = mock_api_call("POST", url, payload, step_name)
session_data = print_call_info("2. UPDATE SESSION (LLM-Selected Shipping)", response, payload)  # Updated title
print(f"[FLOW STATE] New Total: {session_data['totals'][0]['amount']} (LLM selected)")

# ------------------------------------------------------------------------------
# 3. DELEGATE PAYMENT (Generate Secure Token) - Call to Payment Provider
# ------------------------------------------------------------------------------
step_name = "DELEGATE"
url = f"{PAYMENT_PROVIDER_URL}"
payload = ACP_DATA["delegate_payment_request"]
response = mock_api_call("POST", url, payload, step_name)
token_data = print_call_info("3. DELEGATE PAYMENT (Get Secure Token)", response, payload)

SPT_TOKEN = token_data.get('id')
print(f"[FLOW CONTROL] Extracted Scoped Payment Token (SPT): {SPT_TOKEN}\n")

# ------------------------------------------------------------------------------
# 4. COMPLETE CHECKOUT SESSION - Final Call to Merchant (POST /checkout_sessions/{id}/complete)
# ------------------------------------------------------------------------------
step_name = "COMPLETE"
url = f"{BASE_URL}/checkout_sessions/{SESSION_ID}/complete"
# Insert the dynamically obtained SPT into the final request payload
final_payload = copy.deepcopy(ACP_DATA["complete_checkout_session_request_template"])
final_payload["payment_data"]["token"] = SPT_TOKEN

response = mock_api_call("POST", url, final_payload, step_name)
session_data = print_call_info("4. COMPLETE SESSION (Place Order)", response, final_payload)

print("--- ACP Transaction Complete ---")
print(f"Final Status: {session_data.get('status')}")
print(f"Order URL: {session_data.get('order', {}).get('permalink_url')}")

--- Initializing Data Structure ---
[SUCCESS] Loaded data from https://raw.githubusercontent.com/agentic-commerce-protocol/agentic-commerce-protocol/main/examples/examples.agentic_checkout.json and https://raw.githubusercontent.com/agentic-commerce-protocol/agentic-commerce-protocol/main/examples/examples/examples.delegate_payment.json
[SUCCESS] Data Keys available: ['create_checkout_session_request', 'create_checkout_session_response', 'update_checkout_session_request', 'update_checkout_session_response', 'complete_checkout_session_request_template', 'complete_checkout_session_response', 'delegate_payment_request', 'delegate_payment_success_response']
---------------------------------

--- Agentic Commerce Protocol (ACP) Structured Transaction Flow ---

====================== 1. CREATE SESSION (Initialize Cart) ======================
Endpoint: POST https://merchant.example.com/checkout_sessions
Status: 201 Created
--- REQUEST BODY ---
{
  "items": [
    {
      "id": "item_123",
     